In [4]:
import os
import glob

In [5]:
python_config='/home/aurelien/DAWN/DJA_SE++/config/sepp-config.py'
field = 'CEERS'
## Detection image
detect_img=glob.glob(f'/home/aurelien/DAWN/DJA_SE++/image/{field}/cutout/*ir*sci*.fits')[0]
# Filter to use on the detection image
filt='/home/aurelien/DAWN/DJA_SE++/config/gauss_3.0_5x5.conv'

## Output
properties='PixelCentroid,WorldCentroid,SourceIDs,GroupInfo,GroupStamp,SourceFlags,NDetectedPixels,NCorePixel,AperturePhotometry,AutoPhotometry,FluxRadius,SNRRatio,ShapeParameters,FlexibleModelFitting'
name = ".".join(detect_img.split("/")[-1].split(".")[:-1])
output_catname=f'/home/aurelien/DAWN/DJA_SE++/sepp/{field}/{name}_sepp_cat.fits'
checkimg_path=f'/home/aurelien/DAWN/DJA_SE++/sepp/{field}/checkimages'
os.makedirs(checkimg_path, exist_ok=True)

In [6]:
os.system(f"sourcextractor++ \
         --python-config-file {python_config} \
         --output-catalog-filename {output_catname} \
         --output-properties {properties} \
         --detection-image {detect_img} \
         --weight-type WEIGHT \
         --weight-absolute 0 \
         --segmentation-filter {filt} \
         --background-cell-size 128 \
         --segmentation-algorithm LUTZ \
         --smoothing-box-size 5 \
         --detection-threshold 1.15 \
         --detection-minimum-area 6 \
         --partition-corethreshold yes \
         --core-threshold-value 2.25 \
         --core-minimum-area 15 \
         --grouping-algorithm split \
         --partition-multithreshold yes \
         --partition-minimum-area 18 \
         --grouping-hard-limit 0 \
         --partition-minimum-contrast 0.0001 \
         --partition-threshold-count 42 \
         --use-cleaning yes \
         --cleaning-minimum-area 15 \
         --model-fitting-iterations 550 \
         --sampling-scale-factor 1 \
         --check-image-model-fitting {checkimg_path}/model.fits \
         --check-image-residual {checkimg_path}/resid.fits \
         ")


2024-04-23T17:59:38CEST Config  INFO : Loaded segmentation filter: /home/aurelien/DAWN/DJA_SE++/config/gauss_3.0_5x5.conv width: 5 height: 5
2024-04-23T17:59:39CEST Python::stderr  INFO : /home/aurelien/miniforge3/envs/dawn/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
2024-04-23T17:59:39CEST Python::stderr  INFO :   setattr(self, word, getattr(machar, word).flat[0])
2024-04-23T17:59:39CEST Python::stderr  INFO : /home/aurelien/miniforge3/envs/dawn/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
2024-04-23T17:59:39CEST Python::stderr  INFO :   return self._float_to_str(self.smallest_subnormal)
2024-04-23T17:59:40CEST Python::stdout  INFO : F115W
2024-04-23T17:59:40CEST Python::stdout  INFO : F150W
2024-04-23T17:59:40CEST Python::stdout  INFO : F182M
2024-04-23T17:59:40CEST Python::stdout 

0